# Introduction to Probabilistic National Accounts Balancing

## Overview

This notebook introduces the problem of balancing national accounts data and demonstrates why probabilistic programming offers advantages over traditional methods.

### What You'll Learn

1. What is the System of National Accounts (SNA)?
2. Why is data balancing necessary?
3. The traditional RAS method
4. How Gen.jl improves on RAS
5. A worked example with Canadian data

### Prerequisites

- Basic understanding of matrix operations
- Familiarity with Python and NumPy
- No prior knowledge of national accounts or probabilistic programming required!

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# Add src to path
sys.path.insert(0, '../src')

from ras_balancing import ras_balance
from canada_data import (
    simulate_canadian_fof,
    format_matrix_display,
    SECTORS_SHORT,
    INSTRUMENTS_SHORT
)

# Set display options
np.set_printoptions(precision=2, suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_style('whitegrid')

print("✓ Libraries loaded successfully")

## Part 1: The National Accounts Problem

### What is the System of National Accounts?

The **System of National Accounts (SNA)** is like double-entry bookkeeping for an entire economy. It tracks:

- **Production**: What goods and services are produced?
- **Income**: How is income distributed across sectors?
- **Expenditure**: What do people buy?
- **Financial flows**: Who lends to whom?
- **Balance sheets**: Who owns what assets and owes what liabilities?

The key principle: **Everything must balance**. Every transaction appears twice (as a source and a use of funds), and fundamental identities must hold:

$$\text{Production} = \text{Income} = \text{Expenditure}$$ (GDP identity)

$$\sum_{\text{sectors}} (\text{Assets} - \text{Liabilities}) = 0$$ (Financial accounting)

### The Data Quality Problem

Here's the challenge: data comes from multiple sources with varying reliability:

| Data Source | Coverage | Reliability | Examples |
|-------------|----------|-------------|----------|
| **Administrative data** | Incomplete | Very High | Tax records, central bank data |
| **Survey data** | Comprehensive | Medium-Low | Household surveys, business surveys |
| **Derived estimates** | Gap-filling | Variable | Residuals, imputations |

**The problem**: When you compile data from all sources, the accounting identities don't hold!

For example, if you sum up:
- Household wealth from surveys
- Corporate wealth from tax data  
- Government wealth from budget documents
- Foreign wealth from balance of payments

...the total financial assets won't equal total financial liabilities (they should be equal in a closed system, or differ by the current account in an open economy).

### Why This Matters

Unbalanced data leads to:
- Contradictory policy implications
- Loss of user trust
- Inability to use data for economic modeling
- Missed insights (e.g., hidden debt accumulation)

**Example**: If household survey data underreports wealth by 20%, you might think households are poorer than they are, leading to misguided social policy.

## Part 2: A Concrete Example with Canadian Data

Let's look at a simplified flow-of-funds matrix for Canada. This shows holdings of financial assets by sector and instrument type.

In [ ]:
# Generate synthetic Canadian data
np.random.seed(42)
noisy_matrix, true_row_sums, true_col_sums = simulate_canadian_fof(
    scale=2000.0,  # 2 trillion CAD
    noise_level=0.15,  # 15% measurement error
    seed=42
)

# Display the preliminary data
print("\n" + "="*70)
print("PRELIMINARY DATA (Before Balancing)")
print("="*70)
print("\nThis is what we observe from combining different data sources:")
print()

df_noisy = pd.DataFrame(
    noisy_matrix,
    index=SECTORS_SHORT,
    columns=INSTRUMENTS_SHORT[:noisy_matrix.shape[1]]
)

# Add row sums
df_noisy['Row Sum'] = df_noisy.sum(axis=1)
print(df_noisy.to_string())
print()

# Show the imbalance
observed_row_sums = noisy_matrix.sum(axis=1)
observed_col_sums = noisy_matrix.sum(axis=0)

print("\nTarget Row Sums (from administrative data):")
print(f"  {true_row_sums}")
print("\nObserved Row Sums (from preliminary data):")
print(f"  {observed_row_sums}")
print("\nRow Discrepancies:")
print(f"  {observed_row_sums - true_row_sums}")
print(f"\n⚠️  The data doesn't balance! This is the problem we need to solve.")

### Visualizing the Imbalance

In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Row discrepancies
row_errors = observed_row_sums - true_row_sums
colors_row = ['red' if x < 0 else 'green' for x in row_errors]
ax1.barh(SECTORS_SHORT, row_errors, color=colors_row, alpha=0.7)
ax1.axvline(0, color='black', linewidth=0.5)
ax1.set_xlabel('Discrepancy (Billions CAD)', fontsize=11)
ax1.set_title('Row Sum Errors\n(Observed - Target)', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Column discrepancies
col_errors = observed_col_sums - true_col_sums
instruments_used = INSTRUMENTS_SHORT[:len(col_errors)]
colors_col = ['red' if x < 0 else 'green' for x in col_errors]
ax2.bar(instruments_used, col_errors, color=colors_col, alpha=0.7)
ax2.axhline(0, color='black', linewidth=0.5)
ax2.set_ylabel('Discrepancy (Billions CAD)', fontsize=11)
ax2.set_title('Column Sum Errors\n(Observed - Target)', fontsize=12, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Red bars = Underestimated, Green bars = Overestimated")

## Part 3: The Traditional Solution - RAS Method

### What is RAS?

The **RAS algorithm** (named after economists Richard Stone and collaborators) is the standard method for balancing matrices. It works by:

1. **R step**: Scale rows to match target row sums
2. **A step**: (The original matrix - stored but not explicitly in the iteration)
3. **S step**: Scale columns to match target column sums  
4. **Repeat** until convergence

Mathematically:

$$B = \hat{R} \cdot A \cdot \hat{S}$$

where $\hat{R}$ and $\hat{S}$ are diagonal scaling matrices.

### Running RAS on Our Data

In [ ]:
# Run RAS balancing
ras_result = ras_balance(
    noisy_matrix,
    true_row_sums,
    true_col_sums,
    tolerance=1e-6,
    max_iter=1000
)

print("\n" + "="*70)
print("RAS BALANCED MATRIX")
print("="*70)
print()

df_ras = pd.DataFrame(
    ras_result,
    index=SECTORS_SHORT,
    columns=INSTRUMENTS_SHORT[:ras_result.shape[1]]
)
df_ras['Row Sum'] = df_ras.sum(axis=1)
print(df_ras.to_string())

# Verify constraints
print("\nVerification:")
ras_row_sums = ras_result.sum(axis=1)
ras_col_sums = ras_result.sum(axis=0)
print(f"  Max row error: {np.abs(ras_row_sums - true_row_sums).max():.2e}")
print(f"  Max col error: {np.abs(ras_col_sums - true_col_sums).max():.2e}")
print("\n✓ Constraints satisfied!")

### How Did RAS Adjust the Data?

In [ ]:
# Calculate adjustments
ras_adjustments = ras_result - noisy_matrix
ras_pct_adjustments = 100 * ras_adjustments / (noisy_matrix + 1e-10)

# Visualize adjustments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Absolute adjustments
sns.heatmap(
    ras_adjustments,
    annot=True,
    fmt='.1f',
    cmap='RdBu_r',
    center=0,
    xticklabels=INSTRUMENTS_SHORT[:ras_result.shape[1]],
    yticklabels=SECTORS_SHORT,
    ax=ax1,
    cbar_kws={'label': 'Billions CAD'}
)
ax1.set_title('RAS Adjustments (Absolute)', fontsize=12, fontweight='bold')

# Percentage adjustments
sns.heatmap(
    ras_pct_adjustments,
    annot=True,
    fmt='.1f',
    cmap='RdBu_r',
    center=0,
    xticklabels=INSTRUMENTS_SHORT[:ras_result.shape[1]],
    yticklabels=SECTORS_SHORT,
    ax=ax2,
    cbar_kws={'label': 'Percent'}
)
ax2.set_title('RAS Adjustments (Percentage)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("Blue = Decreased, Red = Increased")

## Part 4: The Problem with RAS

### RAS Treats All Data Equally

The fundamental limitation of RAS is that it applies proportional adjustments **without considering data quality**.

Consider:
- Government balance sheet data from official records (very reliable)
- Household cash holdings from surveys (people forget, underreport)

**RAS scales both by the same proportion!** This is suboptimal.

### What We Really Want

Ideally, we want to:

1. **Respect data quality**: Adjust unreliable data more than reliable data
2. **Quantify uncertainty**: Give confidence intervals, not just point estimates
3. **Incorporate domain knowledge**: Use institutional facts (e.g., "banks hold most mortgages")
4. **Detect anomalies**: Flag suspicious data points automatically

This is where **probabilistic programming** with Gen.jl comes in.

## Part 5: The Gen.jl Approach

### Key Idea: Model Data Quality Explicitly

Instead of treating constraints as hard requirements, Gen.jl models them as **probabilistic observations** with uncertainty:

```julia
# In Gen.jl model:
{:row_targets => r} ~ Normal(calculated_row_sum, σ_r)
```

Where:
- **Small σ**: High confidence in this data (e.g., government records)
- **Large σ**: Low confidence (e.g., household surveys)

### The Gen.jl Workflow

1. **Define a generative model**: How is the data produced?
2. **Specify priors**: Start with preliminary estimates
3. **Set uncertainty**: Encode data quality as variance parameters
4. **Run inference**: Find the most likely balanced matrix
5. **Analyze posterior**: Get uncertainty estimates

### Running Gen.jl (Julia Code)

The Gen.jl code is in `src/gen_balancing.jl` and `examples/simple_balance.jl`. To run it:

```bash
julia --project=. examples/simple_balance.jl
```

Or see notebook `03_gen_balancing.ipynb` for interactive Julia examples.

### How Gen.jl Differs from RAS

| Aspect | RAS | Gen.jl |
|--------|-----|--------|
| Data quality | All equal | Explicitly modeled |
| Constraints | Hard (must satisfy) | Soft (weighted by σ) |
| Output | Single matrix | Posterior distribution |
| Adjustments | Proportional | Targeted to high-uncertainty cells |
| Domain knowledge | Limited | Fully customizable |
| Anomaly detection | Manual | Automatic via likelihood |

## Part 6: Comparing Results

Let's compare what RAS and Gen.jl do differently.

### Scenario: Household Wealth Underreporting

Suppose we know:
- Household survey data is unreliable (people underreport wealth)
- Government data is from official budget documents (very reliable)

**Question**: When balancing, should adjustments be concentrated in household data or spread equally?

In [ ]:
# Show where adjustments went
print("\n" + "="*70)
print("WHERE DID RAS PUT THE ADJUSTMENTS?")
print("="*70)

# Calculate total adjustment per sector
adjustment_by_sector = np.abs(ras_adjustments).sum(axis=1)

print("\nTotal Absolute Adjustment by Sector:")
for sector, adj in zip(SECTORS_SHORT, adjustment_by_sector):
    pct = 100 * adj / adjustment_by_sector.sum()
    print(f"  {sector:6s}: {adj:8.2f} CAD billion ({pct:5.1f}% of total adjustments)")

print("\nKey observation:")
print("  RAS spreads adjustments proportionally across all sectors,")
print("  even though we know household data is less reliable than government data.")
print("\n  Gen.jl would concentrate adjustments where uncertainty is highest!")

## Part 7: Next Steps

### Explore More in This Package

1. **`02_ras_method.ipynb`**: Deep dive into RAS algorithm
2. **`03_gen_balancing.ipynb`**: Interactive Gen.jl tutorial
3. **`04_comparison.ipynb`**: Side-by-side RAS vs Gen analysis
4. **`05_uncertainty.ipynb`**: Working with posterior distributions
5. **`06_anomalies.ipynb`**: Detecting outliers and data errors

### Key Takeaways

✓ National accounts data rarely balances due to heterogeneous sources  
✓ RAS is fast and simple but treats all data equally  
✓ Gen.jl allows explicit modeling of data quality  
✓ Probabilistic programming gives uncertainty quantification  
✓ This approach is novel in official statistics (as far as we know!)  

### Further Reading

- **SNA Overview**: `docs/OVERVIEW.md`
- **Gen.jl Documentation**: https://www.gen.dev
- **Statistics Canada**: www23.statcan.gc.ca/imdb/
- **Godley & Lavoie (2007)**: *Monetary Economics* (the SFC bible)

### Questions?

Open an issue on GitHub or explore the documentation!

In [ ]:
print("\n" + "="*70)
print("Thank you for exploring probabilistic national accounts!")
print("="*70)
print("\nRemember: Good accounting is good economics.")
print("          - Wynne Godley")
print()